## FEATURE SELECTION - FootySage

En este cuaderno se define un flujo de trabajo para la selección y análisis de características relevantes utilizando los datos de *Open Data* de ***StatsBomb***. 

Este proceso comienza con la integración de información proveniente de diversas fuentes, como partidos, alineaciones y eventos, consolidándola en un dataset estructurado. Cada fila del dataset representa un partido, enriquecido con métricas calculadas que reflejan aspectos clave del rendimiento, como estadísticas agregadas de pases, disparos, recuperaciones, faltas, entre otros.

A partir de este dataset, se entrenarán modelos básicos de Machine Learning, como Árboles de Decisión (Decision Tree), Regresión Logística (Logistic Regression) y K-Nearest Neighbors (KNN). El objetivo es evaluar su capacidad de clasificación y obtener conclusiones sobre la influencia de las características seleccionadas. Este enfoque inicial proporcionará una base sólida para ampliar el análisis hacia conjuntos de datos más extensos e implementar modelos más avanzados en futuras fases del proyecto.

### Estudio de una competición en una temporada (Competición regular)

#### Carga de datos y preprocesamiento

En primer lugar seleccionamos la competición y la temporada de esa competición que queremos estudiar.

In [7]:
from src.fetch_data import get_competition_id_and_season_id

# seleccionamos el torneo a estudiar (competición, temporada y género)
competition_name = "La Liga"
competition_gender = "male"
season_name = "2015/2016"

competition_id, season_id = get_competition_id_and_season_id(competition_name, competition_gender, season_name)
print(f"We are going to study the tournament {competition_name + ' (' + season_name + ', ' + competition_gender + ')'} whose competition_id={competition_id} and season_id={season_id}")

We are going to study the tournament La Liga (2015/2016, male) whose competition_id=11 and season_id=27


Dada esa competición vamos a obtener cuales son todos los partidos que se han jugado en tal competición esa temporada.

In [90]:
from src.fetch_data import get_matches

matches = get_matches(competition_id, season_id)
print(f"Number of matches in the competition that season: {matches.shape[0]}")

# comprobamos que todos los partidos estén disponibles para obtener información
column_name = "match_status"
if matches[column_name].nunique() == 1: 
    print(f"All values in column '{column_name}' are equal: {matches[column_name].iloc[0]}")
else:
    print(f"The values in column '{column_name}' are different.")

# ordenamos los partidos por semana de competición y hora de inicio para que un futuro cuando
# tengamos que buscar datos de partidos anteriores podamos ver a partir de que momento consultarlo
matches_sorted_by_week = matches.sort_values(by=["match_week","kick_off"])

Number of matches in the competition that season: 380
All values in column 'match_status' are equal: available


C:\Python312\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


Tras tener todos los partidos de la competición ordenados a nuestro gusto, nos disponemos a obtener los datos relacionados con eventos, alineaciones... de cada partido. Estos datos son:
- Número de goles

In [93]:

for _,match in matches_sorted_by_week.iterrows():
    print(match)    
    print("---------------------------")

match_id                                    3825565
match_date                               2015-08-22
kick_off                               18:30:00.000
competition                         Spain - La Liga
season                                    2015/2016
home_team                                  Espanyol
away_team                                    Getafe
home_score                                        1
away_score                                        0
match_status                              available
match_status_360                        unscheduled
last_updated             2022-02-14T13:25:24.237551
last_updated_360                               None
match_week                                        1
competition_stage                    Regular Season
stadium                                RCDE Stadium
referee                           Jesús Gil Manzano
home_managers               Sergio González Soriano
away_managers              Francisco Escriba Segura
data_version